# Cleaning Up the Price Data

Goals:

1.  Working with original `CSV` files
2.  Convert the `item-price` column from a string to a float
3.  Plot to confirm pre-processing worked

In [2]:
import pandas as pd
import os

## Read All CSV Files Into A Dataframe

In [3]:
data_folder = os.path.join('..', 'data', 'waitrose-2024-07')

In [ ]:
# Use list comprehension to collect all the files in the folder and create their folder path
all_files = [
    os.path.join(data_folder, file) for file in os.listdir(data_folder)
    if file.endswith('.csv')

]

In [9]:
for file in all_files:
    print(file)

..\data\waitrose-2024-07\baby-child-and-parent.csv
..\data\waitrose-2024-07\bakery.csv
..\data\waitrose-2024-07\beer-wine-and-spirits.csv
..\data\waitrose-2024-07\best-of-british.csv
..\data\waitrose-2024-07\dietary-and-lifestyle.csv
..\data\waitrose-2024-07\everyday-value.csv
..\data\waitrose-2024-07\food-cupboard.csv
..\data\waitrose-2024-07\fresh-and-chilled.csv
..\data\waitrose-2024-07\frozen.csv
..\data\waitrose-2024-07\home.csv
..\data\waitrose-2024-07\household.csv
..\data\waitrose-2024-07\new.csv
..\data\waitrose-2024-07\organic-shop.csv
..\data\waitrose-2024-07\pet.csv
..\data\waitrose-2024-07\summer.csv
..\data\waitrose-2024-07\tea-coffee-and-soft-drinks.csv
..\data\waitrose-2024-07\toiletries-health-and-beauty.csv
..\data\waitrose-2024-07\waitrose-brands.csv


In [10]:
# Read every single file as a DataFrame
# save data frames as a list
list_of_dfs = [pd.read_csv(file) for file in all_files]

In [11]:
# Concatenate all the files into a single data frame
df = pd.concat(list_of_dfs)

In [12]:
df.head()

,data-product-id,data-product-name,data-product-type,data-product-on-offer,data-product-index,image-url,product-page,product-name,product-size,item-price,price-per-unit,offer-description,category
0,525635,Organix Raspberry & Apple Soft Oaty Bars,G,False,1.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Raspberry & Apple Soft Oaty Bars,6x23g,£3.15,£2.29/100g,NaN,"Baby, Child & Parent"
1,557746,Organix Carrot Cake Oaty Bars,G,False,2.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Carrot Cake Oaty Bars,6x23g,£3.15,£2.29/100g,NaN,"Baby, Child & Parent"
2,32062,Aptamil 2 Follow On Milk,G,False,394.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/aptamil...,Aptamil 2 Follow On Milk,800g,£13.50,£16.88/kg,NaN,"Baby, Child & Parent"
3,767801,Essential Baby Wipes,G,False,4.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Baby Wipes,80s,95p,1.2p each,NaN,"Baby, Child & Parent"
4,514054,Organix Apple Rice Cakes,G,False,5.0,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/organix...,Organix Apple Rice Cakes,40g,£1.60,£4/100g,NaN,"Baby, Child & Parent"


# Puzzle 1: Explore the `item-price` column

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25418 entries, 0 to 1593
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data-product-id        25418 non-null  int64  
 1   data-product-name      25418 non-null  object 
 2   data-product-type      25418 non-null  object 
 3   data-product-on-offer  25418 non-null  bool   
 4   data-product-index     25408 non-null  float64
 5   image-url              25418 non-null  object 
 6   product-page           25418 non-null  object 
 7   product-name           25407 non-null  object 
 8   product-size           25363 non-null  object 
 9   item-price             25407 non-null  object 
 10  price-per-unit         24976 non-null  object 
 11  offer-description      7201 non-null   object 
 12  category               25418 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(10)
memory usage: 2.5+ MB


In [15]:
len(df['item-price'].unique())

775

In [17]:
df['item-price'].head()

0     £3.15
1     £3.15
2    £13.50
3       95p
4     £1.60
Name: item-price, dtype: object

In [29]:
type(df['item-price'].iloc[0])

str

## Cleaning up the Variables

In [52]:
def remove_perunit_tag(price):
    """Remove trailing information about per unit price"""
    if isinstance(price, str):
        return price.strip(' each est.')
    else:
        return price


def remove_currency_lead(price, currency_symbol='£'):
    """Remove leading £"""
    if isinstance(price, str):
        return price.strip(currency_symbol)
    else:
        return price

def remove_trailing_symbol(price, trailing_symbol='p'):
    """Remove trailing value"""
    if isinstance(price, str) and price.endswith(trailing_symbol):
        return '0.' + price.strip(trailing_symbol)
    else:
        return price

In [53]:
df['item-price'] = [remove_perunit_tag(price) for price in df['item-price']]

In [54]:
df['item-price'] = [remove_currency_lead(price) for price in df['item-price']]

In [55]:
df['item-price']

0        3.15
1        3.15
2       13.50
3        0.95
4        1.60
        ...  
1589    24.31
1590     2.40
1591    10.00
1592     2.50
1593    21.49
Name: item-price, Length: 25418, dtype: object

In [56]:
df['item-price'] = df['item-price'].apply(remove_trailing_symbol)

In [57]:
# Examining our missing values

df['item-price'][pd.to_numeric(df['item-price'], errors='coerce').isna()]

291      55-£110
294       30-£38
299       54-£92
302     168-£196
305       31-£40
307      94-£115
311       35-£41
313       32-£39
318       24-£32
48           NaN
49           NaN
48           NaN
49           NaN
3237      42-£60
3279      20-£28
3328     55-£110
3330      62-£79
3360      30-£38
3378      54-£92
3396      60-£80
3435    168-£196
3448      31-£40
3454      25-£38
3471     94-£115
3475      27-£39
3513      35-£41
3536      32-£39
3546      39-£45
3562      24-£32
48           NaN
49           NaN
24           NaN
48           NaN
49           NaN
76        62-£79
1193      42-£60
1218      20-£28
1265     55-£110
1296      30-£38
1320      54-£92
1333      60-£80
1359    168-£196
1375      31-£40
1385      25-£38
1393     94-£115
1397      27-£39
1432      35-£41
1461      39-£45
1466      24-£32
48           NaN
49           NaN
Name: item-price, dtype: object

At this point need to think about what to do with ranges, but can ignore them for now.

In [58]:
df['item-price'] = pd.to_numeric(df['item-price'], errors='coerce')

# Visualising the Pricing Data

In [62]:
import numpy as np
from lets_plot import *
LetsPlot.setup_html()

In [63]:
plot_df = (
    df.groupby('category')['item-price'].describe()
        .reset_index()
        .rename(columns={'25%': 'Q1', '50%': 'median', '75%': 'Q3'})
        .sort_values(by='median')
)

# plot_df.head() to see how it looks like

# This configures what shows up when you hover your mouse over the plot.
tooltip_setup = (
    layer_tooltips()
        .line('@category')
        .line('[@Q1 -- @median -- @Q3]')
        .format('@Q1', '£ {.2f}')
        .format('@median', '£ {.2f}')
        .format('@Q3', '£ {.2f}')
)

g = (
    # Maps the columns to the aesthetics of the plot.
    ggplot(plot_df, aes(y='category', x='median', xmin='Q1', xmax='Q3', fill='category')) +

    # GEOMS

    # Add a line range that 'listens to' columns informed in `ymin` and `ymax` aesthetics
    geom_linerange(size=1, alpha=0.75, tooltips=tooltip_setup) +

    # Add points to the plot (listen to `x` and `y` and fill aesthetics)
    geom_point(size=3, stroke=1, shape=21, tooltips=tooltip_setup) +

    # SCALES

    # Remove the legend (we can already read the categories from the y-axis)
    scale_fill_discrete(guide='none') +

    # Specify names for the axes
    scale_y_continuous(name="Categories\n(from largest to smallest median)", expand=[0.05, 0.05]) +
    scale_x_continuous(name="Price (£)", expand=[0., 0.05], format='£ {.2f}', breaks=np.arange(0, 20, 2.5)) +

    # LABELS
    # It's nice when the plot tells you the key takeaways
    labs(title='"Beer, Wine & Spirits" has the highest median price for individual items',
         subtitle="Dots represent the median price, bars represent the 25th and 75th percentiles") +
    theme(axis_text_x=element_text(size=15),
        axis_text_y=element_text(size=17),
        axis_title_x=element_text(size=20),
        axis_title_y=element_text(size=20),
        plot_title=element_text(size=19, face='bold'),
        plot_subtitle=element_text(size=18),
        legend_position='none') +
    ggsize(1000, 500)

)

g

In [64]:
plot_df.head()

,category,count,mean,std,min,Q1,median,Q3,max
5,Everyday Value,141.0,1.740284,1.202873,0.15,1.10,1.35,2.00,8.00
17,Waitrose Brands,1594.0,3.009454,3.382005,0.10,1.40,2.10,3.40,53.20
1,Bakery,485.0,4.831753,7.728538,0.50,1.60,2.20,3.15,45.00
6,Food Cupboard,4189.0,2.664731,2.138001,0.35,1.60,2.25,3.00,43.50
4,Dietary & Lifestyle,3340.0,3.795734,4.753504,0.10,1.95,2.75,4.00,139.99
